- NaN : 값이 있어야 하는데 없는 결측, 대체, 추정, 예측 등으로 처리
- None : 값이 없는게 값인 결측, 새로운 값으로 정의
- 상황에 따른 처리 방법 선택이 중요

# 삭제

## 행단위 결측 삭제
#### 두가지 조건을 만족하는 경우에만 사용
- 결측이 없는 레코드가 모델을 학습하는데 충분 -> 학습 샘플 개수에 따른 성능의 수렴 여부 확인
- 결측이 새로운 데이터에는 없어야함 -> 도메인 지식 기반의 판단

## 열단위 결측 삭제
#### 두가지 조건을 만족하는 경우에만 사용
- 소수 변수에 결측이 많이 포함되어 있음
- 해당 변수들이 크게 중요하지 않음(by 도메인 지식)
## 관련 문법
- Series/DataFrame.isnull : sum함수와 같이 사용하여 결측치 분포 확인
- DataFrame.dropna : 1이면 열을 삭제, 0이면 행을 삭제, any면 하나라도 결측이면, all면 모든 값이 결측일때 삭제

In [3]:
import pandas as pd
df = pd.read_csv('mammographic.csv')
df.head()

,BI-RADS,Age,Shape,Margin,Density,Output
0,5.0,67.0,3.0,5.0,3.0,1
1,4.0,43.0,1.0,1.0,NaN,1
2,5.0,58.0,4.0,5.0,3.0,1
3,4.0,28.0,1.0,1.0,3.0,0
4,5.0,74.0,1.0,5.0,NaN,1


In [4]:
# 특징과 라벨 분리
X = df.drop('Output', axis = 1)
Y = df['Output']

In [5]:
# 학습데이터와 평가 데이터 분할
from sklearn.model_selection import train_test_split
Train_X , Test_X, Train_Y, Test_Y = train_test_split(X,Y)
Train_X.isnull().sum(axis = 0) # 열별 결측치 개수 확인

BI-RADS     2
Age         3
Shape      24
Margin     34
Density    58
dtype: int64

In [6]:
Train_X.isnull().sum(axis = 0)/len(Train_X) # 열별 결측치 비율 확인
# 한쪽으로 결측이 몰렸다고 할 수 없음

BI-RADS    0.002778
Age        0.004167
Shape      0.033333
Margin     0.047222
Density    0.080556
dtype: float64

In [7]:
# 일단 결측치 행 제거
Train_X.dropna(inplace = True)

c:\users\82106\appdata\local\programs\python\python38\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


### 열 단위 삭제

In [8]:
df = pd.read_csv('post_operative.csv')

In [9]:
x = df.drop('Decision', axis = 1)
y = df['Decision']

In [10]:
train_x, test_x, train_y, test_y = train_test_split(x,y)

In [12]:
train_x.head() # count 변수에 ? 로 결측 있음을 확인
# 이런 문자값은 확인 안 될수가 있으니까 unique를 찍어보는 습관

,L-CORE,L-SURF,L-O2,L-BP,SURF-STBL,CORE-STBL,BP-STBL,COMFORT
14,mid,low,good,high,unstable,unstable,stable,15
6,mid,low,excellent,high,stable,stable,mod-stable,?
61,mid,low,excellent,high,stable,stable,stable,10
30,mid,mid,good,high,unstable,stable,stable,10
78,mid,mid,excellent,mid,unstable,unstable,stable,10


In [13]:
import numpy as np
train_x.replace('?', np.nan).isnull().sum() / len(train_x)
# ? 를 결측치로 처리, 보통 30프로 이상이면 결측치가 많다고 본다

L-CORE       0.000000
L-SURF       0.000000
L-O2         0.000000
L-BP         0.000000
SURF-STBL    0.000000
CORE-STBL    0.000000
BP-STBL      0.000000
COMFORT      0.208955
dtype: float64

In [14]:
train_x.dropna(axis = 1, inplace = True)
test_x.drop('COMFORT', axis = 1, inplace = True)
# test에는 comfort가 결측이 없을 수 있으므로, drop으로 삭제
# train과 test에 속한 컬럼이 다를 수 있다

c:\users\82106\appdata\local\programs\python\python38\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
c:\users\82106\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [15]:
train_x.isnull().sum(axis = 0)

L-CORE       0
L-SURF       0
L-O2         0
L-BP         0
SURF-STBL    0
CORE-STBL    0
BP-STBL      0
COMFORT      0
dtype: int64

# 대체

- 소수 특징에 결측이 쏠린경우와 특징 간 상관성이 큰 경우 활용 부적절
- sklearn.impute.SimpleImputer : 대표값으로 결측을 대체

In [17]:
df = pd.read_csv('./데이터/cleveland.csv')

In [18]:
x = df.drop('Output', axis = 1)
y = df['Output']

In [19]:
train_x, test_x, train_y, test_y = train_test_split(x,y)

In [21]:
train_x.isnull().sum()

Age         0
Sex         0
Cp          0
Trestbps    0
Chol        0
Fbs         0
Restecg     0
Thalach     0
Exang       0
Oldpeak     0
Slope       0
Ca          2
Thal        0
dtype: int64

In [22]:
# 평균 상관 계수 확인 [주의: 모든 변수가 연속형이므로 가능한 접근]
train_x.corr().sum() / (len(train_x.columns)-1)
# 수치가 높지 않다고 판단 -> 특징 간 관계가 크지 않으므로 대체 가능

Age         0.165751
Sex         0.135299
Cp          0.102164
Trestbps    0.125989
Chol        0.109400
Fbs         0.129096
Restecg     0.175147
Thalach    -0.099364
Exang       0.159366
Oldpeak     0.195697
Slope       0.162014
Ca          0.120225
Thal        0.167388
dtype: float64

In [23]:
from sklearn.impute import SimpleImputer

# SimpleImputer 인스턴스화
Si = SimpleImputer(strategy = 'mean')
Si.fit(train_x)

# sklearn instance의 출력은 ndarray이므로 다시 DataFrame으로 바꿔줌
train_x = pd.DataFrame(Si.transform(train_x), columns = train_x.columns)
test_x = pd.DataFrame(Si.transform(test_x), columns = train_x.columns)

In [24]:
train_x.isnull().sum()

Age         0
Sex         0
Cp          0
Trestbps    0
Chol        0
Fbs         0
Restecg     0
Thalach     0
Exang       0
Oldpeak     0
Slope       0
Ca          0
Thal        0
dtype: int64

#### 복잡한 케이스 : 다른 타입의 특징이 있는 경우

In [25]:
df = pd.read_csv('./데이터/saheart.csv')

In [26]:
x = df.drop('Chd', axis = 1)
y = df['Chd']

In [27]:
train_x, test_x, train_y, test_y = train_test_split(x,y)

In [28]:
train_x.isnull().sum()
# 지워도 무방한 수치이지만, 새로 들어온 데이터에 결측이 있을 수도 있다는 도메인

Sbp          0
Tobacco      0
Ldl          0
Adiposity    0
Typea        0
Obesity      9
Alcohol      5
Age          0
Famhist      4
dtype: int64

In [29]:
train_x.corr().sum()/ (len(train_x.columns)-1)
# 수치가 크지 않음

Sbp          0.302438
Tobacco      0.196959
Ldl          0.308117
Adiposity    0.436583
Typea        0.120044
Obesity      0.356907
Alcohol      0.211576
Age          0.387146
Famhist      0.260052
dtype: float64

In [30]:
# Famhist : 범주형 변수
# 그 외 : 연속형 변수

# 대표값을 뭘 해야할지 결정이 어려움 -> 둘 다 사용해야함
# 따라서 데이터 분할
train_x_cate = train_x[['Famhist']]
train_x_cont = train_x.drop('Famhist', axis = 1)

test_x_cate = test_x[['Famhist']]
test_x_cont = test_x.drop('Famhist', axis = 1)

In [32]:
Si_mode = SimpleImputer(strategy = 'most_frequent')
Si_mean = SimpleImputer(strategy = 'mean')

Si_mode.fit(train_x_cate)
Si_mean.fit(train_x_cont)

train_x_cate = pd.DataFrame(Si_mode.transform(train_x_cate), columns  = train_x_cate.columns)
test_x_cate = pd.DataFrame(Si_mode.transform(test_x_cate), columns  = test_x_cate.columns)

train_x_cont = pd.DataFrame(Si_mean.transform(train_x_cont), columns  = train_x_cont.columns)
test_x_cont = pd.DataFrame(Si_mean.transform(test_x_cont), columns  = test_x_cont.columns)

# 두 데이터 붙여주기
train_x = pd.concat([train_x_cate, train_x_cont], axis = 1)
test_x = pd.concat([test_x_cate, test_x_cont], axis = 1)

In [33]:
train_x.isnull().sum()

Famhist      0
Sbp          0
Tobacco      0
Ldl          0
Adiposity    0
Typea        0
Obesity      0
Alcohol      0
Age          0
dtype: int64

In [34]:
# Tip. 이진형 변수와 연속형 변수만 포함된 경우에는 Si_mean만 사용하여 결측치를 대체할 수 있다
# 이진형 변수에 대해서만 round 처리를 하면 하나의 인스턴스만 활용할 수 있다

#### 시계열 변수인 경우 근처값으로 대체 가능
 - DataFrame.fillna : 결측치를 특정 값이나 방법으로 채우는 함수
 - value : 결측치를 대체할 값
 - method : 대체 방법
     - ffill : 결측치 이전의 유효한 값 가운데 가장 가까운 값
     - bfill : 결측치 이후의 유효한 값 가운데 가장 가까운 값
 - train_test_split을 이용하여 임의로 분할한 경우엔 적용 불가능
 - 분할 전에 결측치 대체가 가능한 유일한 케이스

In [35]:
df = pd.read_excel('./데이터/AirQuality.xlsx')
df.isnull().sum()

Date              0
Time              0
CO(GT)            1
PT08.S1(CO)       2
NMHC(GT)          2
C6H6(GT)         10
PT08.S2(NMHC)     4
NOx(GT)           6
PT08.S3(NOx)      7
NO2(GT)           6
PT08.S4(NO2)      6
PT08.S5(O3)       1
T                 0
RH                0
AH                0
dtype: int64

In [36]:
df = df.fillna(method = 'ffill').fillna(method = 'bfill')
# 이전 값을 넣고 뒤값을 넣어줌

In [37]:
df.isnull().sum()

Date             0
Time             0
CO(GT)           0
PT08.S1(CO)      0
NMHC(GT)         0
C6H6(GT)         0
PT08.S2(NMHC)    0
NOx(GT)          0
PT08.S3(NOx)     0
NO2(GT)          0
PT08.S4(NO2)     0
PT08.S5(O3)      0
T                0
RH               0
AH               0
dtype: int64

# 결측치 예측 모델
#### 결측이 발생하지 않은 칼럼을 바탕으로 결측치를 예측하는 모델을 학습하고 활용
- sklearn.impute.KNNImputer : 결측이 아닌 값만을 사용하여 이웃을 두한 뒤, 이웃들의 대표값으로 대체
- n_neighbors : 이웃 수 (5정도가 적절)

### 사용 조건 및 단점
1. 결측이 소수 컬럼에 쏠리면 안된다
2. 특징 간에 관계가 존재해야 한다
3. 다른 결측치 처리 방법에 비해 시간이 오래 소요된다

In [39]:
df = pd.read_csv('./데이터/mammographic.csv')
x = df.drop('Output', axis = 1)
y = df['Output']

train_x, test_x, train_y, test_y = train_test_split(x,y)

In [41]:
train_x.isnull().sum() / len(train_x)

BI-RADS    0.002778
Age        0.005556
Shape      0.027778
Margin     0.041667
Density    0.077778
dtype: float64

In [42]:
train_x.corr().sum() / len(train_x.columns)

BI-RADS    0.421620
Age        0.426653
Shape      0.516058
Margin     0.534703
Density    0.250508
dtype: float64

In [45]:
from sklearn.impute import KNNImputer
KI = KNNImputer(n_neighbors = 5)
KI.fit(train_x)

train_x = pd.DataFrame(KI.transform(train_x), columns = train_x.columns)
test_x = pd.DataFrame(KI.transform(test_x), columns = test_x.columns)

In [46]:
train_x.isnull().sum() / len(train_x)

BI-RADS    0.0
Age        0.0
Shape      0.0
Margin     0.0
Density    0.0
dtype: float64